In [1]:
import pandas as pd
import pickle
import numpy as np
from tqdm import tqdm
import logging
from joblib import Parallel, delayed
import multiprocessing
from multiprocessing import Process, Manager

In [2]:
logging.basicConfig(level=logging.WARNING)

In [3]:
class CategoricalEncoder:
    
    """
    It also changes the data type to int
    """
    
    def __init__(self, cols, method = 'classic'):
        self.COLUMNS = cols
        self.METHOD = method
        self.encoder = {}
        self.inverse_encoder = {}
        
    def fit(self, X, y=None):
        X = X.copy()
        if self.METHOD == 'classic':
            for col in self.COLUMNS:
                self.encoder[col] = {}
                self.inverse_encoder[col] = {}
                unique_values = X[col].unique()
                for i, val in enumerate(unique_values):
                    self.encoder[col][val] = i
                    self.inverse_encoder[col][i] = val
                    
        if self.METHOD == 'mean':
            for col in self.COLUMNS:
                self.encoder[col] = {}
                self.inverse_encoder[col] = {}
                unique_values = X[col].unique()
                for val in unique_values:
                    tmp = X[col][X[col] == val]
                    mean = y.loc[tmp.index, 'P80'].mean()
                    self.encoder[col][val] = mean
                    self.inverse_encoder[col][mean] = val
                self.encoder['NaN'] = y['P80'].mean()
        return self
    
    def transform(self, X):
        X = X.copy()
        for col in self.COLUMNS:
            X[col] = X[col].map(self.encoder[col]).fillna(self.encoder['NaN'])
        return X
    
    def inverse_transform(self, X):
        X = X.copy()
        for col in self.COLUMNS:
            X[col] = X[col].replace(self.inverse_encoder[col])
        return X
    

In [4]:
with open('models/CONF.pickle', 'rb') as handle:
    CONF = pickle.load(handle)
    
with open('models/encoder.pickle', 'rb') as handle:
    encoder = pickle.load(handle)

In [5]:
CATEGORICAL_COLS = ['Fase', 'Tipo de tronadura', 'Tipo Material', 'M', 'Dominio Estructural', 'Tipo Explosivo']
NUMERICAL_COLS = ['Banco', 'Diámetro', 'Fc', 'Cota', 'B', 'S', 'tiempo_1', 'tiempo_2']
TARGET_COLS = ['P10', 'P20', 'P30', 'P40', 'P50', 'P60', 'P70', 'P80', 'P90', 'P100']

In [6]:
def take_in_out(x):
    x = str(x)
    if '..' in x:
        x = x.replace('..','.')
    elif ' in' in x:
        x = x.replace(' in','')
    return x

def take_double_dots_out(x):
    x = str(x)
    if '..' in x:
        x = x.replace('..','.')
    return x

In [7]:
df_stage3 = pd.read_excel('data/Datos_Entregable3_Hackathon.xlsx', header=2)
df_stage3['Norte'] = df_stage3['Norte'].apply(lambda x: take_double_dots_out(x))
df_stage3['Este'] = df_stage3['Este'].apply(lambda x: take_double_dots_out(x))
df_stage3['Norte'] = pd.to_numeric(df_stage3['Norte'].replace({'nan':None}))
df_stage3['Este'] = pd.to_numeric(df_stage3['Este'].replace({'nan':None}))

In [8]:
df_target=df_stage3[TARGET_COLS].copy()
df_target = df_target.add_suffix('_real')
df_stage3=df_stage3.drop(columns=TARGET_COLS,errors='ignore')

In [9]:
#para comparar formato y columnas faltantes
df_historic = pd.read_csv('data/data_fixed.csv', sep = ';')
df_historic = df_historic[df_historic['Tipo Explosivo'] != 'M']
df_historic = df_historic.dropna()

In [10]:
rho_explosivo = {
"A": 0.77,
"B": 1.34,
"C": 1.32,
"D": 1.00,
"E": 1.32,
"F": 1.30,
"G": 1.00,
"H": 1.00,
"I": 1.20,
"J": 1.32,
"K": 1.32,
"L": 1.32
}

rho_roca = {
"L1": 2.53,
"L2": 2.53,
"L3": 2.65,
"L4": 2.65,
"L5": 2.49,
"L6": 2.49,
"L7": 2.65,
"L8": 2.74
}

In [11]:
#se debe convertir la pulgada a metros (dividir por 39.370)
def gr_explosivo(D_p,T,rho_explosivo):
    return (3.14 * (D_p*0.0254)**2) / 4  * ( 16 - T ) * rho_explosivo

def ton_carga(B,S,rho_roca):
    return B*S*15* rho_roca/1000000

In [12]:
diametro=[6,6.5,7.875,9.875,10.625] # se elimina 12.25 debido a que es infactible

In [13]:
B=list(np.arange(3.5,15.1,0.5)) #step de 0.5 en base a lo visto en el historico
B = [ round(elem, 1) for elem in B ]

S=list(np.arange(3.5,15.1,0.5)) #step de 0.5 en base a lo visto en el historico
S = [ round(elem, 1) for elem in S ]

In [14]:
tipo_explosivo=['A','B','C','D','E','F','G','H','I','J','K','L']

In [15]:
T=list(np.arange(1.75,7.6,0.5)) #valores minimos para T segun borde es 1.75, 0.5 para reducir tiempo
T = [ round(elem, 1) for elem in T ]

In [16]:
tiempo_1=list(range(1,20)) #limites establecidos en base al historico
tiempo_2=list(range(1,200)) #limites establecidos en base al historico 
#reminder tiempo_2 volver a valor original

In [17]:
def process_node(i):
    contador=0
    for j in diametro:
        for k in B:
            for l in S:
                for m in tiempo_1:
                    for n in tiempo_2:
                        for o in tipo_explosivo:
                            for p in T:
                                if m>n: # condiciones de borde para tiempo
                                    logging.debug('condicion borde tiempo: tiempo_1 = {m} y tiempo_2 = {n} '.format(m=m,n=n))
                                    continue
                                else:
                                    if (j == 6     and ( (k<3.5 or k>=15) or (l<3.5 or l>=15) ) ):  # condiciones de borde para B y S
                                        logging.debug('condicion borde BxS: diametro = {j}. B = {B} y S = {S}'.format(j=j,B=k,S=l))
                                        continue
                                    elif (j == 6.5   and ( (k<4 or k>=15)   or (l<4   or l>=15) ) ):  # condiciones de borde para B y S
                                        logging.debug('condicion borde BxS: diametro = {j}. B = {B} y S = {S}'.format(j=j,B=k,S=l))
                                        continue
                                    elif (j == 7.875 and ( (k<4 or k>=15)   or (l<4   or l>=15) ) ):  # condiciones de borde para B y S
                                        logging.debug('condicion borde BxS: diametro = {j}. B = {B} y S = {S}'.format(j=j,B=k,S=l))
                                        continue
                                    elif (j == 9.875 and ( (k<5 or k>=15)   or (l<5   or l>=15) ) ):  # condiciones de borde para B y S
                                        logging.debug('condicion borde BxS: diametro = {j}. B = {B} y S = {S}'.format(j=j,B=k,S=l))
                                        continue 
                                    elif (j == 10.625and ( (k<5.5 or k>=15) or (l<5.5 or l>=15) ) ):  # condiciones de borde para B y S
                                        logging.debug('condicion borde BxS: diametro = {j}. B = {B} y S = {S}'.format(j=j,B=k,S=l))
                                        continue
                                    elif (j == 12.25 and ( (k<5.5 or k>=15) or (l<5.5 or l>=15) ) ):  # condiciones de borde para B y S
                                        logging.debug('condicion borde BxS: diametro = {j}. B = {B} y S = {S}'.format(j=j,B=k,S=l))
                                        continue
                                    else: # cumple con las condiciones minimas de B y S y tiempo
                                        if 25*j*0.0254>p or k/2>p or l/2>p or 7.5<p:  # condiciones de borde para el TACO
                                            logging.debug('condicion borde TACO = {}'.format(p))
                                            continue
                                        else: # cumple con las condiciones del TACO
                                            if gr_explosivo(j,p,rho_explosivo[o])/ton_carga(k,l,rho_roca[df_stage3.iloc[i]['M']])>1000 or gr_explosivo(j,p,rho_explosivo[o])/ton_carga(k,l,rho_roca[df_stage3.iloc[i]['M']])<100: #condiciones de FC
                                                logging.debug('condicion borde FC  = {}'.format(gr_explosivo(j,p,rho_explosivo[o])/ton_carga(k,l,rho_roca[df_stage3.iloc[i]['M']])))
                                                continue
                                            else: # cumple con condiciones FC
                                                temp = {
                                                "index": int(i),
                                                "Diámetro": float(j),
                                                "B": float(k),
                                                "S": float(l),
                                                "tiempo_1": int(m),
                                                "tiempo_2": int(n),
                                                "Fc": float(gr_explosivo(j,p,rho_explosivo[o])/ton_carga(k,l,rho_roca[df_stage3.iloc[i]['M']])),
                                                "Tipo Explosivo": o
                                                }
                                                malla_list.append(temp)
                                                contador=contador+1
                                                if contador%10==0:
                                                    return 'done' #return de example, hay que borrar para que corra sobre todas las posibilidades
    return 'malla finalizada'

    

In [18]:
malla_list = Manager().list()  # <-- can be shared between processes.
malla = pd.DataFrame(columns=[*CATEGORICAL_COLS, *NUMERICAL_COLS,*TARGET_COLS])
Parallel(n_jobs=-1)(delayed(process_node)(i) for i in tqdm(range(df_stage3.shape[0])))

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 64.70it/s]


['done',
 'done',
 'done',
 'done',
 'done',
 'done',
 'done',
 'done',
 'done',
 'done',
 'done']

In [19]:
malla_frame=pd.DataFrame(list(malla_list))

In [20]:
malla_frame.set_index('index',inplace=True)

In [21]:
malla=df_stage3.join(malla_frame)

In [22]:
df_test = encoder.transform(malla)
for target in TARGET_COLS:
    tmp = df_test[CONF[target]['columns']]
    predictions = CONF[target]['model'].predict(tmp)
    df_test[target] = predictions

In [23]:
final = df_test.join(df_target)

In [24]:
final['dif_80'] = abs((final['P80_real'] - final['P80'])/final['P80_real'])*100
final['dif_70'] = abs((final['P70_real'] - final['P70'])/final['P70_real'])*100

In [25]:
final[(final['dif_80']<10) | (final['dif_70']<10)]

,Fase,Banco,Tipo de tronadura,Tipo Material,M,Dominio Estructural,Este,Norte,Cota,B,...,P30_real,P40_real,P50_real,P60_real,P70_real,P80_real,P90_real,P100_real,dif_80,dif_70
0,5.068701,2795,5.068701,5.068701,5.803466,4.677983,58999.00,90277.0,2795,3.5,...,1.850395,2.401576,3.188978,3.976380,4.881892,6.023625,7.795280,14.566937,1.941817,3.811842
0,5.068701,2795,5.068701,5.068701,5.803466,4.677983,58999.00,90277.0,2795,3.5,...,1.850395,2.401576,3.188978,3.976380,4.881892,6.023625,7.795280,14.566937,1.941817,3.533809
0,5.068701,2795,5.068701,5.068701,5.803466,4.677983,58999.00,90277.0,2795,3.5,...,1.850395,2.401576,3.188978,3.976380,4.881892,6.023625,7.795280,14.566937,1.941817,3.533809
0,5.068701,2795,5.068701,5.068701,5.803466,4.677983,58999.00,90277.0,2795,3.5,...,1.850395,2.401576,3.188978,3.976380,4.881892,6.023625,7.795280,14.566937,1.941817,3.533809
0,5.068701,2795,5.068701,5.068701,5.803466,4.677983,58999.00,90277.0,2795,3.5,...,1.850395,2.401576,3.188978,3.976380,4.881892,6.023625,7.795280,14.566937,1.941817,3.861551
0,5.068701,2795,5.068701,5.068701,5.803466,4.677983,58999.00,90277.0,2795,3.5,...,1.850395,2.401576,3.188978,3.976380,4.881892,6.023625,7.795280,14.566937,1.941817,3.861551
0,5.068701,2795,5.068701,5.068701,5.803466,4.677983,58999.00,90277.0,2795,3.5,...,1.850395,2.401576,3.188978,3.976380,4.881892,6.023625,7.795280,14.566937,1.941817,3.861551
0,5.068701,2795,5.068701,5.068701,5.803466,4.677983,58999.00,90277.0,2795,3.5,...,1.850395,2.401576,3.188978,3.976380,4.881892,6.023625,7.795280,14.566937,0.659489,5.527481
0,5.068701,2795,5.068701,5.068701,5.803466,4.677983,58999.00,90277.0,2795,3.5,...,1.850395,2.401576,3.188978,3.976380,4.881892,6.023625,7.795280,14.566937,0.659489,5.527481
0,5.068701,2795,5.068701,5.068701,5.803466,4.677983,58999.00,90277.0,2795,3.5,...,1.850395,2.401576,3.188978,3.976380,4.881892,6.023625,7.795280,14.566937,0.659489,7.312065
